In [ ]:
!hdfs dfs -ls /

In [9]:
!sudo hdfs dfs -mkdir -p /user/hadoop-repo


We trust you have received the usual lecture from the local System
Administrator. It usually boils down to these three things:

    #1) Respect the privacy of others.
    #2) Think before you type.
    #3) With great power comes great responsibility.

[sudo] password for alice: 


In [10]:
testpass

NameError: name 'testpass' is not defined

In [ ]:
!hdfs --config ~/hadoop_palmetto/config dfs -put

In [ ]:
!hdfs --config ~/hadoop_palmetto/config dfs -put text hadoop-repo

In [ ]:
!hdfs --config ~/hadoop_palmetto/config fsck hadoop-repo/text/complete-shakespeare.txt -files -blocks -locations

In [ ]:
!mkdir codes

In [ ]:
!pwd

In [ ]:
!hdfs dfs -cat hadoop-repo/text/complete-shakespeare.txt \
    2>/dev/null | head -n 100

In [ ]:
%%writefile codes/wordcountMapper.py

#!/software/spackages/linux-centos8-x86_64/gcc-8.3.1/anaconda3-2019.10-v5cuhr6keyz5ryxcwvv2jkzfj2gwrj4a/bin/python   

import sys                                                                                                
for oneLine in sys.stdin:
    oneLine = oneLine.strip()
    for word in oneLine.split(" "):
        if word != "":
            print ('%s\t%s' % (word, 1))
        

In [ ]:
!hdfs --config ~/hadoop_palmetto/config dfs -cat hadoop-repo/text/complete-shakespeare.txt \
    2>/dev/null \
    | head -n 20 \
    | python ./codes/wordcountMapper.py
    | sort


In [ ]:
%%writefile codes/wordcountReducer.py
#!/software/spackages/linux-centos8-x86_64/gcc-8.3.1/anaconda3-2019.10-v5cuhr6keyz5ryxcwvv2jkzfj2gwrj4a/bin/python
import sys

current_word = None
total_word_count = 0

for line in sys.stdin:
    line = line.strip()
    word, count = line.split("\t", 1)
    try:
        count = int(count)
    except ValueError:
        continue
    
    if current_word == word:
        total_word_count += count
    else:
        if current_word:
            print ("%s\t%s" % (current_word, total_word_count))
        current_word = word
        total_word_count = 1
        
if current_word == word:
    print ("%s\t%s" % (current_word, total_word_count))

In [ ]:
!hdfs --config ~/hadoop_palmetto/config dfs -cat hadoop-repo/text/complete-shakespeare.txt \
    2>/dev/null \
    | head -n 20 \
    | python ./codes/wordcountMapper.py
    | sort \
        | python ./codes/wordcountReducer.py


In [ ]:
!hdfs --config ~/hadoop_palmetto/config dfs -rm -R hadoop-repo/output-wordcount

!mapred --config ~/hadoop_palmetto/config streaming \
    -input intro-to-hadoop/text/complete-shakespeare.txt \
    -output hadoop-repo/output-wordcount \
    -file ./codes/wordcountMapper.py \
    -mapper wordcountMapper.py \
    -file ./codes/wordcountReducer.py \
    -reducer wordcountReducer.py

In [ ]:
!hdfs dfs -ls hadoop-repo/output-wordcount

In [ ]:
!hdfs dfs -cat hadoop-repo/output-wordcount/part-00000 \
    2>/dev/null | head -n 100

In [ ]:
%%writefile codes/wordcountEnhancedMapper.py
#!/software/spackages/linux-centos8-x86_64/gcc-8.3.1/anaconda3-2019.10-v5cuhr6keyz5ryxcwvv2jkzfj2gwrj4a/bin/python                                          
import sys                     
import string

translator = str.maketrans('', '', string.punctuation)

for oneLine in sys.stdin:
    oneLine = oneLine.strip()
    for word in oneLine.split(" "):
        if word != "":
            newWord = word.translate(translator).lower()
            print ('%s\t%s' % (_______, 1))

In [ ]:
!hdfs dfs -rm -R hadoop-repo/output-wordcount-enhanced

!ssh dsciutil yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
    -input hadoop-repo/text/gutenberg-shakespeare.txt \
    -output hadoop-repo/output-wordcount \
    -file ____________________________________________________ \
    -mapper _____________________ \
    -file ____________________________________________________ \
    -reducer _____________________ \

In [ ]:
Debug jobs

In [ ]:
Dataset
http://files.grouplens.org/datasets/movielens/ml-10m-README.html

In [ ]:
!hdfs dfs -ls -h /repository/movielens

In [ ]:
Find movies which have the highest average ratings over the years and report their ratings and genres
Find the average ratings of all movies over the years
Sort the average ratings from highest to lowest
Report the results, augmented by genres

In [ ]:
!hdfs dfs -ls /repository/movielens
!hdfs dfs -cat /repository/movielens/README.txt

!hdfs dfs -cat /repository/movielens/links.csv \
    2>/dev/null | head -n 5

!hdfs dfs -cat /repository/movielens/movies.csv \
    2>/dev/null | head -n 5

!hdfs dfs -cat /repository/movielens/ratings.csv \
    2>/dev/null | head -n 5

!hdfs dfs -cat /repository/movielens/tags.csv \
    2>/dev/null | head -n 5

In [ ]:
%%writefile codes/avgRatingMapper.py
#!/usr/bin/env python

import sys
import csv 

movieFile = "./movielens/movies.csv"
movieList = {}


with open(movieFile, mode = 'r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        movieList[row[0]] = {}
        movieList[row[0]]["title"] = row[1]
        movieList[row[0]]["genre"] = row[2]

for oneMovie in sys.stdin:
    oneMovie = oneMovie.strip()
    ratingInfo = oneMovie.split(",")
    try:
        movieTitle = movieList[ratingInfo[1]]["title"]
        movieGenre = movieList[ratingInfo[1]]["genre"]
        rating = float(ratingInfo[2])
        print ("%s\t%s\t%s" % (movieTitle, rating, movieGenre))
    except ValueError:
        continue

In [ ]:
!hdfs dfs -cat /repository/movielens/ratings.csv \
    2>/dev/null | head -n 10 | python ./codes/avgRatingMapper.py

In [ ]:
!mkdir movielens
!mkdir movielens
!hdfs dfs -get /repository/movielens/movies.csv movielens/movies.csv


In [ ]:
Test Reducer 


In [ ]:
Optimization
